In [1]:
import os, sys, time, copy
import numpy as np
import matplotlib.pyplot as plt
import pickle

sys.path.append('../')
sys.path.append('../Lib')
sys.path.append('../Models')
sys.path.append('../Protocols')
from cell_models import kernik, protocols, paci_2018

import mod_protocols
import protocol_lib
import mod_kernik as kernik
import mod_trace as trace
from Models.br1977 import BR1977
from ord2011 import ORD2011
import model_response

C:\Anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
def get_trace(model, protocol, prestep):
    prestep_protocol = protocol_lib.VoltageClampProtocol([protocol_lib.VoltageClampStep(voltage=-80.0, duration=prestep)])
    model.generate_response(prestep_protocol, is_no_ion_selective=False)
    model.y_ss = model.y[:, -1]
    response_trace = model.generate_response(protocol, is_no_ion_selective=False)
    return response_trace


def get_long_protocol(individual_dictionary, holding_step):
    all_steps = []
    holding_step = protocol_lib.VoltageClampStep(-80, holding_step)
    
    for current, protocol in individual_dictionary.items():
        all_steps.append(holding_step)
        all_steps += protocol.steps
        
    long_protocol = protocol_lib.VoltageClampProtocol(all_steps)
    return long_protocol




def remove_start_of_protocol(protocol, removal_time_step):
    vc_segment_endpoints = protocol.get_voltage_change_endpoints()

    is_found = False
    i = 0
    while not is_found:
        if removal_time_step < vc_segment_endpoints[i]:
            max_segment_idx = i
            is_found = True
        i += 1

    new_start_voltage = protocol.get_voltage_at_time(removal_time_step)
    new_duration = (vc_segment_endpoints[max_segment_idx] - removal_time_step)

    #TODO make separate function and call from remove_end..()
    if isinstance(protocol.steps[max_segment_idx], protocol_lib.VoltageClampRamp) or isinstance(protocol.steps[max_segment_idx], mod_protocols.VoltageClampRamp):
        new_final_voltage = protocol.steps[max_segment_idx].voltage_end
        new_segment = protocol_lib.VoltageClampRamp( new_start_voltage, new_final_voltage, new_duration)
    else:
        new_start_voltage = protocol.steps[max_segment_idx].voltage
        new_segment = protocol_lib.VoltageClampStep( new_start_voltage, new_duration)

    new_protocol = protocol_lib.VoltageClampProtocol( [new_segment] + protocol.steps[(max_segment_idx + 1):])

    return new_protocol





def shorten_protocol_start(protocol, start_time, prestep, window, step_size, max_contribution,
                           current_name, acceptable_change, model_name, removal_time_step=200,
                           scale=1):
    
    min_acceptable_current = max_contribution * acceptable_change
    
    last_protocol = None
    while (max_contribution > min_acceptable_current).values[0]:
        if protocol.get_voltage_change_endpoints()[-1] <= removal_time_step:
            return protocol

#         print(max_contribution)
        last_protocol = copy.copy(protocol)
        
        protocol = remove_start_of_protocol( protocol, removal_time_step=removal_time_step)

        max_currents = get_max_currents( protocol, prestep=prestep, window=window, step_size=step_size, scale=scale, model_name=model_name)
        max_contribution = max_currents[max_currents["Current"]==current_name]["Contribution"]

    return last_protocol




def get_protocol_without_end(protocol, start_time, window, extra_time, scale=1):
    window = window/scale
    extra_time = extra_time/scale

    vc_segment_endpoints = protocol.get_voltage_change_endpoints()

    cutoff_time = scale * (start_time + extra_time)

    if cutoff_time >  vc_segment_endpoints[-1]:        
        return protocol

    is_found = False
    i = 0
    while not is_found:
        if cutoff_time < vc_segment_endpoints[i]:
            max_segment_idx = i
            is_found = True
        i += 1
       
    new_duration = (cutoff_time - vc_segment_endpoints[max_segment_idx-1])
    if new_duration <= 0.0:
        print("Warning ......................................................")

    if isinstance(protocol.steps[max_segment_idx], protocol_lib.VoltageClampRamp) or isinstance(protocol.steps[max_segment_idx], mod_protocols.VoltageClampRamp):
        new_start_voltage = protocol.steps[max_segment_idx].voltage_start
        new_final_voltage = protocol.get_voltage_at_time(cutoff_time)
        new_segment = protocol_lib.VoltageClampRamp( new_start_voltage, new_final_voltage, new_duration)
    else:
        new_start_voltage = protocol.steps[max_segment_idx].voltage
        new_segment = protocol_lib.VoltageClampStep( new_start_voltage, new_duration)

     
    new_protocol = protocol_lib.VoltageClampProtocol( protocol.steps[0:max_segment_idx] + [new_segment])

    return new_protocol



def get_max_currents(vc_protocol, prestep, window, step_size, model_name, scale=1):
    if model_name == 'Paci':
        baseline_paci= paci_2018.PaciModel(is_exp_artefact=True)
        i_trace = get_trace(baseline_paci, vc_protocol, prestep=prestep)
    elif model_name == 'BR1977':
        model = BR1977(vc_protocol)
        i_trace = model_response.get_model_response_JK(model, vc_protocol, prestep=prestep)        
    elif model_name == 'ORD2011':
        model = ORD2011(vc_protocol)
        i_trace = model_response.get_model_response_JK(model, vc_protocol, prestep=prestep)        
    elif model_name == 'OHara2017':
        model = '../mmt-model-files/ohara-cipa-v1-2017_VC.mmt'
        i_trace = model_response.get_model_response_with_myokit(model, vc_protocol)
    else:
        baseline_kernik = kernik.KernikModel(is_exp_artefact=True)
        i_trace = get_trace(baseline_kernik, vc_protocol, prestep=prestep)        
        
    max_currents = i_trace.current_response_info.get_max_current_contributions(
            i_trace.t, window=window/scale, step_size=step_size/scale)

    return max_currents


def shorten_protocol(best_individual, current_name, only_end, model_name, window=10, step_size=5, prestep=2000):
    vc_protocol = best_individual.protocol
    length_of_protocol = vc_protocol.get_voltage_change_endpoints()[-1]

    scale = 1

    max_currents = get_max_currents(vc_protocol, prestep=prestep, window=window, step_size=step_size, 
                                    model_name=model_name, scale=scale)    
#     print(max_currents[max_currents["Current"]==current_name])     # 0    I_Na      0.976547       860.0     870.0
    start_time = float(max_currents[max_currents["Current"]==current_name]["Time Start"])
    shortened_protocol = get_protocol_without_end( vc_protocol, start_time, window, extra_time=100, scale=scale)

    if not only_end:
        max_contribution = max_currents[max_currents["Current"]==current_name]["Contribution"]
        if current_name == "I_Kr" or "IKr" or "ikr.IKr":
            accepted_threshold = .99
        else: 
            accepted_threshold = .95
        shortened_protocol = shorten_protocol_start(shortened_protocol, start_time, prestep, window, step_size, max_contribution,
                                                    current_name, acceptable_change=accepted_threshold, scale=scale,
                                                    model_name=model_name)

    print(
        f'Protocol length of {current_name} decreased from {length_of_protocol} to {shortened_protocol.get_voltage_change_endpoints()[-1]}.')
    return shortened_protocol



def get_high_fitness(ga_result):
    best_individual = ga_result.generations[0][0]

    for i, gen in enumerate(ga_result.generations):
        best_in_gen = ga_result.get_high_fitness_individual(i)
        if best_in_gen.fitness > best_individual.fitness:
            best_individual = best_in_gen

    return best_individual


def make_shortened_results(trial_conditions, only_end, 
                           holding_step, prestep, window, step_size, 
                           with_artefact=False, model_name='ORD2011', 
                           currents=['I_Na', 'I_NaL', 'I_to', 'I_CaL', 'I_Kr', 'I_Ks', 'I_K1' ] ):
        
    folder = f"ga_results/{trial_conditions}"

    original_protocols = {}
    shortened_protocols = {}

    for current in currents:
        ga_result = pickle.load(open(f'{folder}/ga_results_{current}_artefact_{with_artefact}', 'rb'))
        best_individual = get_high_fitness(ga_result)        
        shortened_protocols[current] = shorten_protocol(best_individual, 
                                                        window=window, 
                                                        prestep=prestep,
                                                        current_name=current,
                                                        only_end=only_end, 
                                                        model_name=model_name)
        original_protocols[current] = best_individual.protocol
        
        pickle.dump(shortened_protocols[current], open(f"{folder}/short_{current}_p{prestep}_oe{only_end}_a{with_artefact}.pkl", 'wb'))

    new_long_protocol = get_long_protocol(shortened_protocols, holding_step)
    scale = 1
    shortened_max_currents = get_max_currents( new_long_protocol, prestep=prestep, window=window, step_size=step_size,
                                               model_name=model_name, scale=scale)

    print(f"The shortened_max_currents are {shortened_max_currents}")
    shortened_max_currents.to_csv(
            f"{folder}/{trial_conditions}_h{holding_step}_p{prestep}_oe{only_end}_a{with_artefact}.csv")

    pickle.dump(new_long_protocol, open(f"{folder}/{trial_conditions}_h{holding_step}_p{prestep}_oe{only_end}_a{with_artefact}.pkl", 'wb'))
    
#     return shortened_protocols, new_long_protocol

In [3]:
trial_conditions = "ORD2011_288_51_4_-121_61"
prestep = 5000
window = 10
step_size = 5
holding_step = 500
only_end = False
with_artefact =False
model_name = trial_conditions.split('_')[0]

currents = ['I_Na', 'I_Kr', 'I_Ks', 'I_To', 'I_CaL', 'I_K1', 'I_NaL' ]   
if model_name=='BR1977':
    with_artefact = True
    currents = ['I_Na', 'I_si', 'I_K1', 'I_x1']
elif model_name=='Kernik':        
    with_artefact = True
    currents = ['I_Na', 'I_Kr', 'I_Ks', 'I_To', 'I_F', 'I_CaL', 'I_K1'] 

start_time = time.time()
make_shortened_results(trial_conditions, only_end=only_end, holding_step=holding_step, 
                       prestep=prestep, window=window, step_size=step_size,
                       with_artefact=with_artefact, model_name=model_name, currents=currents)
print("--- %s seconds ---"%(time.time()-start_time))

Protocol length of I_Na decreased from 1863.5334859662971 to 160.00000000000009.
Protocol length of I_Kr decreased from 2932.0736275209306 to 732.0736275209306.
Protocol length of I_Ks decreased from 1947.3674095165165 to 1885.0.
Protocol length of I_To decreased from 1238.2278817983574 to 250.0.
Protocol length of I_CaL decreased from 3327.8708441918725 to 995.0.
Protocol length of I_K1 decreased from 2818.910161581648 to 200.0.
Protocol length of I_NaL decreased from 2776.347622663437 to 1576.347622663437.
The shortened_max_currents are       Current  Contribution  Time Start  Time End
0        I_Na      0.959381       560.0     570.0
1       I_NaL      0.334883      9225.0    9235.0
2        I_To      0.651772      4930.0    4940.0
3       I_CaL      0.766935      6425.0    6435.0
4      I_CaNa      0.082823       625.0     635.0
5       I_CaK      0.162097      1700.0    1710.0
6        I_Kr      0.746330      1795.0    1805.0
7        I_Ks      0.476356      4175.0    4185.0
8    